<a href="https://colab.research.google.com/github/vjhawar12/FreshNET-A-mobileNET-adaptation/blob/main/FreshNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip -q install lightning-bolts

In [1]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset
from tqdm import tqdm
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from copy import deepcopy
from torch.ao.quantization import QuantStub, DeQuantStub, prepare_qat, get_default_qat_qconfig, convert, fuse_modules

/usr/local/lib/python3.11/dist-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/usr/local/lib/python3.11/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.11/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:92: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality ma

# Loading dataset from Google Cloud Storage

In [15]:
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=GSR4ESjldk7xd8bEnHANy3zILxt7qY&prompt=consent&token_usage=remote&access_type=offline&code_challenge=P5btcZDVKD1-I5Silp08YC1TJVSpZ4fUOzqBzJz_pEE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJhFRpjFHfD0TPGKTTRwyhfPFAXcB7TEtPdJKaqp46-kQFkfALnCFhecSEP1foKJOg

You are now logged in as [vedantj1203@gmail.com].
Your current project 

In [16]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=h9JRgbYNEBmExPwN9GYLxZXdKLeByu&prompt=consent&token_usage=remote&access_type=offline&code_challenge=6FNGRmV674yAwDEnyg2mo2Caehu2TEQfrlxn5Zm9bSA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJhoSXjxZrpDpiCmkoIiICg_GNIeGf-WgpEEOe6ACcXKCQ627iPLZUFPi_Wq50o2OQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Q

In [17]:
!gcloud config set project freshnet-466505

Updated property [core/project].


In [18]:
def download_bucket_with_transfer_manager(bucket_name, destination_directory="", workers=8, max_results=1000):

    from google.cloud.storage import Client, transfer_manager

    client = Client()
    bucket = client.bucket(bucket_name)

    blob_names = [blob.name for blob in bucket.list_blobs(max_results=max_results)]

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))

download_bucket_with_transfer_manager("freshnet-images", destination_directory="/content/dataset/", workers=8, max_results=None)

Failed to download test/ due to exception: [Errno 21] Is a directory: '/content/dataset/test/'
Failed to download test/fresh/ due to exception: [Errno 21] Is a directory: '/content/dataset/test/fresh/'
Downloaded test/fresh/test_fresh_images.zip to /content/dataset/test/fresh/test_fresh_images.zip.
Failed to download test/rotten/ due to exception: [Errno 21] Is a directory: '/content/dataset/test/rotten/'
Downloaded test/rotten/test_rotten_images.zip to /content/dataset/test/rotten/test_rotten_images.zip.
Failed to download train/ due to exception: [Errno 21] Is a directory: '/content/dataset/train/'
Failed to download train/fresh/ due to exception: [Errno 21] Is a directory: '/content/dataset/train/fresh/'
Downloaded train/fresh/train_fresh_images.zip to /content/dataset/train/fresh/train_fresh_images.zip.
Failed to download train/rotten/ due to exception: [Errno 21] Is a directory: '/content/dataset/train/rotten/'
Downloaded train/rotten/train_rotten_images.zip to /content/dataset/tr

In [19]:
!cd /content/dataset/train/rotten && unzip -j train_rotten_images.zip

Archive:  train_rotten_images.zip
replace p_r367.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [20]:
!cd /content/dataset/train/fresh && unzip -j train_fresh_images.zip

Archive:  train_fresh_images.zip
replace b_f882.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [21]:
!cd /content/dataset/test/fresh && unzip -j test_fresh_images.zip

Archive:  test_fresh_images.zip
replace b_f304.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [22]:
!cd /content/dataset/test/rotten && unzip -j test_rotten_images.zip

Archive:  test_rotten_images.zip
replace p_r367.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Hyperparameters

In [43]:
BATCH_SIZE = 256
RANDOM_SEED = 42
TRAIN_SIZE, VAL_SIZE = 0.8, 0.2
IMG_SIZE = 320 # Original images are ~400*400 px so resizing them to 320 retains detail while reducing computational cost
EPOCHS = 30
WARMUP_DUR = 10 # num of warmup epochs
LR_MIN = 0.0001 # minimum learning rate
MILD_DROPOUT_RATE = 0.10 # mild enough to avoid overfitting on a small dataset

# Preprocessing

In [44]:
"""
    Mild preprocessing only. No MixUp, CutMix, RandomCrop, or ColorJitter because a lightweight model like this one
    is less likely to overfit. Also, this model needs to perform fine-grained classification,
    so aggressive augmentations could distort the small image regions (like signs of fungi or discoloration)
    that are crucial for accurate prediction.
"""

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation([-180, 180]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading data into Dataloader

In [25]:
path_to_train_imgs = "/content/dataset/train"
path_to_test_imgs = "/content/dataset/test"

full_train_data = ImageFolder(path_to_train_imgs, transform=train_transform)
test_data = ImageFolder(path_to_test_imgs, transform=test_transform)
train_data, val_data = random_split(full_train_data, [TRAIN_SIZE, VAL_SIZE], generator=torch.Generator().manual_seed(RANDOM_SEED))

print(f"{full_train_data.class_to_idx} \n {test_data.class_to_idx}")

{'fresh': 0, 'rotten': 1} 
 {'fresh': 0, 'rotten': 1}


In [26]:
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=10)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

indices_to_keep = [i for i, (image, label) in enumerate(test_data) if i < 20]
mini = Subset(test_data, indices_to_keep)
mini_test_dataloader = DataLoader(mini, batch_size=5, shuffle=False, num_workers=4)

indices_to_keep = [i for i, (image, label) in enumerate(test_data) if i > 20 and i < 40]
calibrate = Subset(test_data, indices_to_keep)
calibrate_dataloader = DataLoader(calibrate, shuffle=False, num_workers=4)

# CUDA optimizations

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  torch.backends.cuda.matmul.allow_tf32 = True
  torch.backends.cudnn.allow_tf32 = True
  torch.backends.cuda.enable_flash_sdp(True)
  torch.backends.cuda.enable_mem_efficient_sdp(True)
  torch.backends.cuda.enable_math_sdp(True)

# Quantization helper class: Float Wrapper

This class wraps operations that Pytorch cannot quantize, such as deptwise convolutions with groups != 1, with a DeQuantStub and a QuantStub. This ensures during quantization the particular operations wrapped by a FloatWrapper instance receive float32 objects and not INT8.

In [28]:
class FloatWrapper(nn.Module):
  def __init__(self, layer):
    super().__init__()
    self.layer = layer
    self.quant = QuantStub()
    self.dequant = DeQuantStub()
    self.qconfig = None
    self.layer.qconfig = None

  def forward(self, x):
    x = self.dequant(x)
    x = self.layer(x)
    x = self.quant(x)

    return x

# Depthwise Seperable Convolution implementation

In [29]:
class Block(nn.Sequential):
  def __init__(self, in_channels, out, out_channels, kernel_size, downsample_factor):
    super().__init__(
      # Fusing these 3
      nn.Conv2d(in_channels=in_channels, out_channels=out, kernel_size=1, stride=1, groups=1), # expansion
      nn.BatchNorm2d(out),
      nn.ReLU(),

      FloatWrapper(
          # adding padding because the image downsamples, but skip connections require same size
          nn.Conv2d(in_channels=out, out_channels=out, kernel_size=kernel_size, padding=kernel_size // 2, stride=downsample_factor, groups=out)
          # depthwise
      ),

      # Fusing these 3
      nn.Conv2d(in_channels=out, out_channels=out_channels, kernel_size=1, stride=1, groups=1), # pointwise
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
    )


In [30]:
class DepthwiseSeperableConvolution(nn.Module):
  def __init__(self, in_channels, out_channels, exp_factor, downsample_factor, kernel_size=3):
    super().__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.exp_factor = exp_factor
    self.downsample_factor = downsample_factor
    self.kernel_size = kernel_size

    out = self.in_channels * self.exp_factor

    self.preserve = self.in_channels == self.out_channels
    """
        batchnorm normalizes inputs which has shown to improve training. It's typically applied after the convolutional layer output and before the activation function
        The MobileNET paper also mentions, with the exception of the final fully connected linear layer, "all layers are followed by a batchnorm and ReLU nonlinearity"
    """


    self.block = Block(self.in_channels, out, self.out_channels, self.kernel_size, self.downsample_factor)

    """
    nn.Sequential(
        nn.Conv2d(in_channels=self.in_channels, out_channels=out, kernel_size=1, stride=1, groups=1), # expansion
        nn.BatchNorm2d(out),

        FloatWrapper(
            # adding padding because the image downsamples, but skip connections require same size
            nn.Conv2d(in_channels=out, out_channels=out, kernel_size=self.kernel_size, padding=self.kernel_size // 2, stride=self.downsample_factor, groups=out)
            # depthwise
        ),

        nn.BatchNorm2d(out),
        nn.Conv2d(in_channels=out, out_channels=self.out_channels, kernel_size=1, stride=1, groups=1), # pointwise
        nn.BatchNorm2d(self.out_channels),

        nn.ReLU(),
    )
    """

  def forward(self, x):
    for layer in self.block:
      x = layer(x)

    return x


# Skip connection

In [52]:
"""
FreshNET only applies skip connections between DepthwiseSeperableConvolution instances which preserve channel dimension in order to adhere to the laws of vector addition.
"""
class SkipConnection(nn.Module):
  def __init__(self, seq): # seq is an nn.Sequential instance
    super().__init__()
    self.quant = QuantStub()
    self.dequant = DeQuantStub()
    self.seq = seq

    for module in seq:
      if not hasattr(module, 'preserve'):
        raise Exception("Cannot apply skip connection between layers of different channel dimensions")

    self.seq = seq

  def forward(self, x):
    # addition is not supported with quantized tensors so we dequant, perform addition, then quant again
    x = self.dequant(x)
    self.gradient = x

    for depthwise_sep_conv in self.seq:
      x = depthwise_sep_conv(x)

    x = self.quant(x + self.gradient)
    return x

# FreshNet: A MobileNET adaptation

In [53]:
# A MobileNET adaptation
class FreshNET(nn.Module):

  """
  FreshNET applies the concept of depthwise seperable convolutions as mentioned in the MobileNet paper, but the image dimensions are slightly increased
  to better suit the dataset and leverage higher image quality. Specifically, FreshNET decouples spatial filtering and channel mixing,
  the two operations traditionally combined in standard convolutions, and performs them as separate, more efficient operations. As the authors of MobileNet noted,
  it "drastically reduc[es] computation and model size". Despite this computational efficiency, accuracy is largely preserved -- the original MobileNet paper reports
  only about a 1% drop in accuracy compared to standard convolutions.

  This separation is implemented in DepthwiseSeperableConvolution class. Each instance consists of:
  - A pointwise 1*1 convolution for channel expansion, increasing feature dimensionality.
  - A depthwise 3*3 convolution applied independently per channel for spatial filtering.
  - Another pointwise convolution to project back to a lower-dimensional space.

  This design significantly reduces computational cost while maintaining high representational power. There are 10 layers, of which 7 are DepthwiseSeperableConvolution layers.
  Each DepthwiseSeperableConvolution layer consists of 1-4 DepthwiseSeperableConvolution instances. Within the DepthwiseSeperableConvolution layer, there are non-linear activation functions.

  Skip connections are also applied where possible to improve gradient flow and preserve fine-grained information. For a task like fresh/rotten classification,
  fine-grained information can be very valuable.
  """

  def __init__(self):
    super().__init__()

    # QuantStub and DeQuantStub are used to go from FL32 <===> quantized
    self.quant = QuantStub()
    self.dequant = DeQuantStub()

    # SiLU is good for vanishing gradients and can be applied in the later layers where ReLU might not be as effective (dead neurons)
    silu = nn.SiLU()

    # Regularization
    """
    The original paper notes "we use less regularization and data augmentation techniques because small models have less trouble with overfitting".
    This is why I chose to use dropout with 15% probability since that's on the lower end of the probability spectrum.
    """
    dropout_mild = nn.Dropout(MILD_DROPOUT_RATE)

    # initial regular convolution
    block_0 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),

    ) # 320*320*3 --> 160*160*32

    # inverted residual block 1: 160*160*32 --> 160*160*16, channel expansion = 1
    block_1 = nn.Sequential(
        DepthwiseSeperableConvolution(32, 16, 1, 1)
    )

    # inverted residual block 2: 160*160*16 --> 80*80*24, channel expansion = 6
    block_2 = nn.Sequential(
        DepthwiseSeperableConvolution(16, 24, 6, 2),
        DepthwiseSeperableConvolution(24, 24, 6, 1)
    )

    # inverted residual block 3: 80*80*24 --> 40*40*32, channel expansion = 6
    block_3 = nn.Sequential(
        DepthwiseSeperableConvolution(24, 32, 6, 2),
        SkipConnection(nn.Sequential(
              DepthwiseSeperableConvolution(32, 32, 6, 1),
              DepthwiseSeperableConvolution(32, 32, 6, 1),
          )
        )
    )

    # inverted residual block 4: 40*40*32 --> 20*20*64, channel expansion = 6
    block_4 = nn.Sequential(
        DepthwiseSeperableConvolution(32, 64, 6, 2),
        SkipConnection(nn.Sequential(
              DepthwiseSeperableConvolution(64, 64, 6, 1),
              DepthwiseSeperableConvolution(64, 64, 6, 1),
              DepthwiseSeperableConvolution(64, 64, 6, 1),
            )
        )
    )

    # inverted residual block 5: 20*20*64 --> 20*20*96, channel expansion = 6
    block_5 = nn.Sequential(
        DepthwiseSeperableConvolution(64, 96, 6, 1),
        SkipConnection(nn.Sequential(
              DepthwiseSeperableConvolution(96, 96, 6, 1),
              DepthwiseSeperableConvolution(96, 96, 6, 1),
            )
        )
    )

    # inverted residual block 6: 20*20*96 --> 10*10*160, channel expansion = 6
    block_6 = nn.Sequential(
        DepthwiseSeperableConvolution(96, 160, 6, 2),
        SkipConnection(nn.Sequential(
              DepthwiseSeperableConvolution(160, 160, 6, 1),
              DepthwiseSeperableConvolution(160, 160, 6, 1),
            )
        )
    )

    # inverted residual block 7: 10*10*160 --> 10*10*320, channel expansion = 6
    block_7 = nn.Sequential(
        DepthwiseSeperableConvolution(160, 320, 6, 1)
    )

    # final regular convolution
    final_conv = nn.Conv2d(in_channels=320, out_channels=1280, kernel_size=1, stride=1) # 10*10*320 --> 10*10*1280

    """
      MaxPooling is often preferred for highlighting dominant features, but it can be too aggressive in lightweight models
      like MobileNet or EfficientNet, which already have low spatial resolution. It can discard valuable spatial
      information. Average Pooling computes the average of nearby pixels, preserving more context. This makes it more
      suitable for compact architectures like this one.
    """

    avg_pool = nn.AvgPool2d(10) # 10*10*1280 --> 1*1*1280
    flatten = nn.Flatten() # fc layers expect a single tensor not a feature map

    # fully connected layers
    """
    The MobileNET paper mapped from 1280 directly to 1000 since it was being trained on ImageNET. My dataset, however, only has 2
    output classes: fresh (0) or rotten (1). Mapping directly from 1280 to 2 is an abrupt jump which could limit the model's ability to
    distinguish between classes, so I introduced an additional layer to map from 1280 to 500 then applied nonlinearity and
    mild dropout before going from 500 to 2.
    """
    fc_1 = nn.Linear(in_features=1280, out_features=500)
    fc_2 = nn.Linear(in_features=500, out_features=2)

    """
    The sequence of layers is as follows:

      - intial regular convolution
      - 4 stacks of DepthwiseSeperableConvolution
      - mild dropout for regularization
      - 3 more stacks of DepthwiseSeperableConvolution
      - 1 final regular convolution
      - average pooling
      - activation function
      - first fully connected linear layer
      - activation function
      - mild dropout for regularization
      - final fully connected linear layer

    There are skip connections in blocks 3-6.
    """

    self.layers = nn.Sequential(
        block_0,

        block_1,
        block_2,
        block_3,
        block_4,
        FloatWrapper(dropout_mild),

        block_5,
        block_6,
        block_7,

        final_conv,
        avg_pool,
        flatten,
        FloatWrapper(silu),
        FloatWrapper(dropout_mild),

        fc_1,
        FloatWrapper(silu),

        fc_2
    )

  # fusing conv + bn + relu improves speed and efficiency during QAT
  def fuse(self):
    for module_name, module in self.named_modules(): # iterating over the instances defined in __init__ of this class
      if isinstance(module, Block):
        fuse_modules(module, [["0", "1", "2"], ["4", "5", "6"]], inplace=True)
      elif module_name == "layers.0":
        fuse_modules(module, [["0", "1", "2"]], inplace=True)


  def forward(self, x):
    x = self.quant(x)

    for layer in self.layers:
      x = layer(x)

    x = self.dequant(x)

    return x

# Setup

In [54]:
cnn = FreshNET()
cnn.to(device)

loss_fn = nn.CrossEntropyLoss() # std loss function for classification

optimizer = optim.Adam(cnn.parameters()) # RMSProp is pretty sensitive to changes in LR so i wanted to try Adam

scaler = torch.amp.GradScaler()

"""
Not mentioned in the original paper, but this scheduler was used because linear warmup reduces volatility in the earlier epochs.
Cosine annealing can improve training stability and convergence and declining LR steadily means the model will rely more on the features
it learns early on -- the key differentiators -- rather than picking up potential noise and overfitting.
"""
scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=WARMUP_DUR, eta_min=LR_MIN, max_epochs=EPOCHS)

/tmp/ipython-input-96917922.py:15: UnderReviewWarning: The feature LinearWarmupCosineAnnealingLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=WARMUP_DUR, eta_min=LR_MIN, max_epochs=EPOCHS)


# Train and validate

In [55]:
def train(cnn, epochs=EPOCHS, scaler=None):
  cnn.train()

  for i in range(epochs):
    torch.cuda.empty_cache()
    correct, total = 0, 0
    running_loss = 0

    cnn.train(True)

    loop = tqdm(train_dataloader, desc=f"Epoch {i+1}/{epochs}", leave=True, disable=False)

    for j, (input, labels) in enumerate(loop, 1):
      input, labels = input.to(device), labels.to(device)
      optimizer.zero_grad()

      output = cnn(input)
      loss = loss_fn(output, labels)
      running_loss += loss.item()

      if scaler:
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
      else:
        loss.backward()
        optimizer.step()

      avg_loss = running_loss / j
      loop.set_postfix(loss=avg_loss)

      pred = torch.argmax(output, dim=1)

      total += labels.size(0)
      correct += (pred == labels).sum().item()


    cnn.eval()
    val_total, val_correct = 0, 0

    with torch.no_grad():
      for image, label in val_dataloader:
        image, label = image.to(device), label.to(device)
        output = cnn(image)

        pred = torch.argmax(output, dim=1)
        val_total += label.size(0)
        val_correct += (pred == label).sum().item()

      val_accuracy = val_correct / val_total
      accuracy = correct / total


    scheduler.step()

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {i + 1}: LR={current_lr:.6f} \t Train Acc: {accuracy:.4f} \t Val Acc: {val_accuracy:.4f}")

In [56]:
cnn.to("cuda")
compiled_cnn = torch.compile(cnn)

with torch.autocast(device_type='cuda', dtype=torch.float16):
  train(compiled_cnn, scaler=scaler)

Epoch 1/30: 100%|██████████| 74/74 [01:02<00:00,  1.18it/s, loss=0.695]


Epoch 1: LR=0.000111 	 Train Acc: 0.4526 	 Val Acc: 0.4641


Epoch 2/30:   8%|▊         | 6/74 [00:09<01:45,  1.55s/it, loss=0.695]


KeyboardInterrupt: 

# Saving model weights

In [ ]:
cnn.to("cpu")
torch.save(cnn.state_dict(), "/content/modelweight.pth")

from google.colab import files
files.download(r"/content/modelweight.pth")

# Loading saved model

In [ ]:
from google.colab import files
#files.upload()
weights = torch.load('/content/modelweight.pth', map_location=torch.device('cpu'))
cnn = FreshNET()
cnn.load_state_dict(weights)

# Model size (no quantization)

In [ ]:
param_size = 0

for param in cnn.parameters():
    param_size += param.nelement() * param.element_size()

buffer_size = 0

for buffer in cnn.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

# Final test accuracy (no quantization)

In [ ]:
def test(test_dataloader, cnn):
  with torch.no_grad():
    cnn.eval()
    accuracy = 0
    total = 0

    for batch in test_dataloader:
      images, labels = batch

      images = images.to("cpu")
      labels = labels.to("cpu")

      if images.dtype != torch.float32:
        raise Exception("Pre-Quantization invalid data type")

      output = cnn(images)

      if images.dtype != torch.float32:
        raise Exception("Post-Quantization invalid data type")

      predicted = torch.argmax(output, dim=1)
      accuracy += (predicted == labels).sum().item()
      total += labels.size(0)

    avg_acc = accuracy / total

    return avg_acc


cnn.eval()

test_acc = test(test_dataloader, cnn)
print(f"Final test accuracy: {100 * test_acc:.2f}%")



In [ ]:
cnn.to("cuda")
cnn.eval()

with torch.autocast(device_type='cuda', dtype=torch.float16):
  test_acc = test(test_dataloader, cnn)
  print(f"Final test accuracy: {100 * test_acc:.2f}%")

# Quantizing model

Quantization Aware Training (QAT) involves training the model with full floating point precision, float32, while simulating the effects of int8 quantization. This 'fake' quantization makes the model well-equipped to perform under quantization, maintaining accuracy better than if it used post-training  quantization instead.


In [36]:
cnn.to("cpu") # model must be on cpu while fusing
cnn.eval()
cnn.fuse() # custom fuse method fuses the DepthwiseSeperableConvolution layers like conv2d, batchnorm, and relu
cnn.to("cpu") # need to switch back to cpu before quantizing
config = get_default_qat_qconfig("qnnpack") # for ARM-based architecture, qnnpack is the correct config mode
cnn.qconfig = config
cnn.train()
prepare_qat(cnn, inplace=True) # preparing the model for QAT
cnn.eval()


FreshNET(
  (quant): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=255, qscheme=torch.per_tensor_affine, reduce_range=False
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (dequant): DeQuantStub()
  (layers): Sequential(
    (0): Sequential(
      (0): ConvReLU2d(
        3, 32, kernel_size=(3, 3), stride=(2, 2)
        (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
          fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_tensor_symmetric, reduce_range=False
          (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
        )
        (activation_post_proces

In [37]:
# calibrating model before converting to get meaningful zero_point and scale values

with torch.no_grad():
    for batch in calibrate_dataloader:
        images, _ = batch
        images = images.to("cpu")
        output = cnn(images)


for name, module in cnn.named_modules():
    if hasattr(module, "activation_post_process"):
        obs = module.activation_post_process
        if hasattr(obs, "calculate_qparams"):
            scale, zero_point = obs.calculate_qparams()
            print(f"Using observer from: {name}")
            break

scale = scale.item()
zero_point = zero_point.item()

print(scale)
print(zero_point)

Using observer from: quant
0.01688021793961525
99


In [38]:
quantized_cnn = convert(cnn, inplace=False)
quantized_cnn.to("cpu")

FreshNET(
  (quant): Quantize(scale=tensor([0.0169]), zero_point=tensor([99]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (layers): Sequential(
    (0): Sequential(
      (0): QuantizedConvReLU2d(3, 32, kernel_size=(3, 3), stride=(2, 2), scale=0.0204971544444561, zero_point=0)
      (1): Identity()
      (2): Identity()
    )
    (1): Sequential(
      (0): DepthwiseSeperableConvolution(
        (block): Block(
          (0): QuantizedConvReLU2d(32, 32, kernel_size=(1, 1), stride=(1, 1), scale=0.02337159588932991, zero_point=0)
          (1): Identity()
          (2): Identity()
          (3): FloatWrapper(
            (layer): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (quant): QuantStub()
            (dequant): DeQuantStub()
          )
          (4): QuantizedConvReLU2d(32, 16, kernel_size=(1, 1), stride=(1, 1), scale=0.023689599707722664, zero_point=0)
          (5): Identity()
          (6): Identity()
        )
      )
    )
    

In [39]:
for name, module in quantized_cnn.named_modules():
    if isinstance(module, torch.nn.quantized.Conv2d):
      if module.bias() is not None:
        dataType = module.bias().dtype
        print(f"{name} bias dtype: {dataType}")

layers.0.0 bias dtype: torch.float32
layers.1.0.block.0 bias dtype: torch.float32
layers.1.0.block.4 bias dtype: torch.float32
layers.2.0.block.0 bias dtype: torch.float32
layers.2.0.block.4 bias dtype: torch.float32
layers.2.1.block.0 bias dtype: torch.float32
layers.2.1.block.4 bias dtype: torch.float32
layers.3.0.block.0 bias dtype: torch.float32
layers.3.0.block.4 bias dtype: torch.float32
layers.3.1.seq.0.block.0 bias dtype: torch.float32
layers.3.1.seq.0.block.4 bias dtype: torch.float32
layers.3.1.seq.1.block.0 bias dtype: torch.float32
layers.3.1.seq.1.block.4 bias dtype: torch.float32
layers.4.0.block.0 bias dtype: torch.float32
layers.4.0.block.4 bias dtype: torch.float32
layers.4.1.seq.0.block.0 bias dtype: torch.float32
layers.4.1.seq.0.block.4 bias dtype: torch.float32
layers.4.1.seq.1.block.0 bias dtype: torch.float32
layers.4.1.seq.1.block.4 bias dtype: torch.float32
layers.4.1.seq.2.block.0 bias dtype: torch.float32
layers.4.1.seq.2.block.4 bias dtype: torch.float32
lay

# Model size (with quantization)


In [ ]:
param_size = 0

for param in quantized_cnn.parameters():
    param_size += param.nelement() * param.element_size()

buffer_size = 0

for buffer in quantized_cnn.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

# Final test accuracy (with quantization)

In [ ]:
quantized_cnn.eval()
quantized_cnn.to("cuda")
test_acc = test(test_dataloader, quantized_cnn)
print(f"Final test accuracy of quantized model: {100 * test_acc:.2f}%")